# Spaces

Spaces are handled in `ebcc` according to the following nomenclature:

                ─┬─ ┌──────────┐
                 │  │  frozen  │
                 │  ├──────────┤ ─┬─
         virtual │  │  active  │  │
                 │  ├──────────┤  │ correlated
                 │  │ inactive │  │
                ─┼─ ├══════════┤ ─┼─
                 │  │ inactive │  │
                 │  ├──────────┤  │ correlated
        occupied │  │  active  │  │
                 │  ├──────────┤ ─┴─
                 │  │  frozen  │
                ─┴─ └──────────┘

A plain coupled cluster calculation will operate within the correlated space. The simplest use of this system is frozen core calculations.

First, we again find system and mean-field using PySCF, and initialise an appropriate logger.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(atom="H 0 0 0; F 0 0 1.1", basis="cc-pvdz", verbose=0)
mf = scf.RHF(mol).run()

In [2]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

The `Space` class is constructed by providing boolean arrays indicating whether an orbital at that index is occupied, frozen, or active. In this example we freeze the two lowest energy MOs.

In [3]:
from ebcc import Space

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True

space = Space(occupied, frozen, active)

print("Space:", space)

Space: (5o, 14v) [(2o, 0v) frozen]


The `active` space is used for methods that differentiate between the correlated orbitals in order to treat a subset of orbitals at a higher level of theory. The `frozen` and `active` arrays must be disjoint.

In [4]:
occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:1] = True
active[np.sum(mf.mo_occ > 0) - 1] = True
space = Space(occupied, frozen, active)
print("Space:", space)

try:
    active = frozen
    space = Space(occupied, frozen, active)
except ValueError as e:
    print("Error:", e)

Space: (5o, 14v) [(1o, 0v) frozen, (1o, 0v) active]
Error: Frozen and active orbitals must be mutually exclusive.


The space can be used in a calculation to perform the frozen-core CC calculation. All methods should be compatible with this procedure, as the code generation is agnostic to the definition of the space.

In [5]:
from ebcc import REBCC

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True
space = Space(occupied, frozen, active)

cc2 = REBCC(mf, space=space, log=log)
cc2.kernel()

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.4
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (5o, 14v) [(2o, 0v) frozen]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.1547617180    -100.1421591584
   1    -0.1561846863    -100.1435821267     1.423e-03     8.569e-03
   2    -0.1578343707    -100.1452318110     1.650e-03     4.811e-03
   3    -0.1583435249    -100.1457409653     5.092e-04     2.697e-03
   4    -0.1584821749    -100.1458796152     1.386e-04     8.051e-04
   5    -0.1584614174    -100.1458588577     2.076e-05     2.810e-04
   6    -0.1584609032 

-0.15846102874067258

An example of the use of the so-called active space is CCSDt', where the third-order amplitudes span only the active orbitals. We can mix this with frozen orbitals to make use of all three tiers of the space. In this example we freeze the two lowest energy orbitals, and allow the third-order amplitudes to span the two highest energy occpued and two lowest energy virtual MOs.

In [6]:
occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
active[np.sum(mf.mo_occ > 0) - 1] = True
active[np.sum(mf.mo_occ > 0) - 2] = True
active[np.sum(mf.mo_occ > 0)    ] = True
active[np.sum(mf.mo_occ > 0) + 1] = True
frozen[:2] = True
space = Space(occupied, frozen, active)

ccsdt = REBCC(mf, ansatz="CCSDt'", space=space, log=log)
ccsdt.kernel()


RCCSDt'
*******

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSDt'

Space: (5o, 14v) [(2o, 0v) frozen, (2o, 2v) active]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.1547617180    -100.1421591584
   1    -0.1561846863    -100.1435821267     1.423e-03     8.569e-03
   2    -0.1578343707    -100.1452318110     1.650e-03     4.811e-03
   3    -0.1583435249    -100.1457409653     5.092e-04     2.697e-03
   4    -0.1584821749    -100.1458796152     1.386e-04     8.051e-04
   5    -0.1584614174    -100.1458588577     2.076e-05     2.810e-04
   6    -0.1584609032    -100.1458583436     5.142e-07     1.771e-05
   7    -0.1584611299    -100.1458585703     2.267e-07     2.799e-06
   8    -0.1584610258    -100.1458584662     1.041e-07     3.111e-07
   9    -0.1584610274    -100.1458584677     1.555e-09     2.979e-08
  10    -0.1584610287    -1

-0.15846102874067253

Frozen natural orbital (FNO) calculations can be easily performed using the helper function to construct the space.

In [7]:
from ebcc.ham.space import construct_fno_space

no_coeff, no_occ, no_space = construct_fno_space(mf, occ_tol=1e-3)

fno_ccsd = REBCC(mf, mo_coeff=no_coeff, mo_occ=no_occ, space=no_space, log=log)
fno_ccsd.kernel()


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (5o, 14v) [(0o, 4v) frozen]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.2056399556    -100.1930373960
   1    -0.2060870504    -100.1934844908     4.471e-04     1.236e-02
   2    -0.2092752058    -100.1966726462     3.188e-03     8.115e-03
   3    -0.2103493295    -100.1977467698     1.074e-03     4.600e-03
   4    -0.2105330257    -100.1979304661     1.837e-04     9.277e-04
   5    -0.2105101223    -100.1979075627     2.290e-05     1.806e-04
   6    -0.2105056240    -100.1979030643     4.498e-06     2.100e-05
   7    -0.2105067621    -100.1979042024     1.138e-06     5.501e-06
   8    -0.2105066317    -100.1979040721     1.303e-07     5.015e-07
   9    -0.2105066182    -100.1979040586     1.349e-08     4.546e-08
  10    -0.2105066237    -100.1979040640     5.439

-0.21050662310756654

External corrections are also supported, and use the designated active space to signify the orbitals in which the external amplitudes are calculated.

In [8]:
from pyscf import fci, ao2mo
from ebcc.ext.fci import extract_amplitudes_restricted

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
active[np.sum(mf.mo_occ > 0) - 4 : np.sum(mf.mo_occ > 0) + 4] = True
space = Space(occupied, frozen, active)

mo = mf.mo_coeff[:, space.active]
h1e = np.einsum("pq,pi,qj->ij", mf.get_hcore(), mo, mo)
h2e = ao2mo.kernel(mf._eri, mo, compact=False).reshape((mo.shape[-1],) * 4)
ci = fci.direct_spin1.FCI()
ci.kernel(h1e, h2e, space.nact, space.naocc * 2)
amplitudes = extract_amplitudes_restricted(ci, space)

eccc = REBCC(mf, ansatz="CCSD", space=space, log=log)
eccc.external_correction(amplitudes, mixed_term_strategy="update")


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (5o, 14v) [(4o, 4v) active]

Applying external corrections.
 > mixed_terms_strategy:  fixed

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.2113674613    -100.1987649016
   1    -0.2115608690    -100.1989583093     1.934e-04     1.186e-02
   2    -0.2151752884    -100.2025727287     3.614e-03     6.161e-03
   3    -0.2159914112    -100.2033888515     8.161e-04     3.152e-03
   4    -0.2164134044    -100.2038108448     4.220e-04     1.455e-03
   5    -0.2163828573    -100.2037802977     3.055e-05     3.599e-04
   6    -0.2163733547    -100.2037707951     9.503e-06     6.534e-05
   7    -0.2163749626    -100.2037724030     1.608e-06     1.970e-05
   8    -0.2163750135    -100.2037724539     5.086e-08     2.379e-06
   9    -0.2163748945    -100.2037723349     1.190e-07 

-0.21637490567254425